In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import re

BASE_SEED = 5142024
rng = np.random.default_rng(BASE_SEED)

# Floating point safe vectorized comparison function
W = lambda T_pi1, T_pi2: (~np.isclose(T_pi1, T_pi2) & (T_pi1 > T_pi2)) + 0.5 * np.isclose(T_pi1, T_pi2)

########################################################
# Read in the data and process it into a usable form
########################################################

raw_df = pd.read_csv("./MY-LC-Data/41586_2023_6651_MOESM4_ESM.csv")

df = raw_df.copy()
df = df[df["x0_Censor_Complete"] == 0] # Remove censored patients
df["LC"] = (df["x0_Censor_Cohort_ID"] == 3).astype(np.float32) # Retrieve Long Covid Status

# Select and rename the features of interest
cell_features_names = list(filter(lambda x: re.search(r"Flow_Cyt_.*ML$", x), df.columns))
other_features = [
    "LC",
    "x0_Demographics_Age",
    "x0_Demographics_Sex",
    "x0_Demographics_BMI",
]
all_features = other_features + cell_features_names

def feature_renaming(name):
    parts = name.split("_")
    if (len(parts) == 1):
        return(parts[0])
    elif (len(parts) == 3):
        return(parts[2])
    else:
        return(parts[3])
df = df[all_features].rename(feature_renaming, axis = "columns")

cell_features_names = list(map(feature_renaming, cell_features_names))
demo_features_names = list(map(feature_renaming, other_features[1:]))

# Create the design matrix
df = df.dropna(subset = ["Age", "Sex", "BMI"]) # First drop any rows that have missing demographics

df["Sex"] = df["Sex"] - 1 # Convert Sex to {0, 1} rather than {1, 2}
df["Age*BMI"] = df["Age"] * df["BMI"]
df["Sex*BMI"] = df["Sex"] * df["BMI"]

# Do not standardize X
X_unfiltered = df[["LC"]].to_numpy()

# Standardize Z
Z_unfiltered = df[["Age", "Sex", "BMI", "Age*BMI", "Sex*BMI"]].to_numpy()
Z_unfiltered = (Z_unfiltered - Z_unfiltered.mean(axis = 0)) / Z_unfiltered.std(axis = 0)
Z_unfiltered = np.concatenate([np.ones((Z_unfiltered.shape[0], 1)), Z_unfiltered], axis = 1)

feature_names_dict = {}
feature_names_dict["DC1ofLive"] = "cDC1"
feature_names_dict["ncMonoofLive"] = "Nonclassical monocytes"
feature_names_dict["cMonoofLive"] = "Classical monocytes"
feature_names_dict["TotalNeutofLive"] = "Total neutrophils"
feature_names_dict["CD16hiNeutofLive"] = "CD16hi neutrophils"
feature_names_dict["CD16intNeutofLive"] = "CD16int neutrophils"
feature_names_dict["CD8TofCD3"] = "T cell(cytotoxic)"
feature_names_dict["NaiveCD8TofCD8"] = "CD8+ T(naive)"
feature_names_dict["FollCD8ofCD8"] = "CD8+ T cell(follicular)"
feature_names_dict["CD8TexofCD8"] = "CD8+ T cell(exhuasted)"
feature_names_dict["CD8TCRactofCD8"] = "CD8+ T cell(activated TCR)"
feature_names_dict["PD1pCD8TemraofCD8"] = "CD8+ T cell(PD1+Temr)"
feature_names_dict["CD8TemofCD8"] = "CD8+ T cell(Tem)"
feature_names_dict["CD8PD1pTcmofCD8"] = "CD8+ T cell(PD1+Tcm)"
feature_names_dict["CD8PD1pAAofCD8"] = "CD8+ T cell(PD1+AA)"
feature_names_dict["CD8TemraofCD8"] = "CD8+ T cell(Temra)"
feature_names_dict["CD4PD1pTcmofCD4"] = "CD4+ T cell(PD1+Tcm)"
feature_names_dict["CD4IL4pofCD4"] = "CD4+ T cell(IL4+)"
feature_names_dict["CD4TcmofCD4"] = "CD4+ T cell(Tcm)"
feature_names_dict["CD4TemofCD4"] = "CD4+ T cell(Tem)"
feature_names_dict["CD4TexofCD4"] = "CD4+ T cell(exhuasted)"
feature_names_dict["CD4PD1pAAofCD4"] = "CD4+ T cell(PD1+AA)"
feature_names_dict["CD4TofCD3"] = "T cell(helper)"
feature_names_dict["CD4TCRactofCD4"] = "CD4+ T cell(activated TCR)"
feature_names_dict["CD8CXCR3pofCD8"] = "CD8+ T cell(CXCR3+)"
feature_names_dict["CD8GranzymeBpofCD8"] = "CD8+ T cell(GranzymeB+)"
feature_names_dict["CD8TNFapofCD8"] = "CD8+ T cell(TNFa+)"
feature_names_dict["CD8IFNypofCD8"] = "CD8+ T cell(IFNy+)"
feature_names_dict["CD4CXCR3pofCD4"] = "CD4+ T cell(CXCR3+)"
feature_names_dict["CD4IL17pofCD4"] = "CD4+ T cell(IL17+)"
feature_names_dict["CD4TNFapofCD4"] = "CD4+ T cell(TNFa+)"
feature_names_dict["CD4IFNypofCD4"] = "CD4+ T cell(IFNy+)"
feature_names_dict["CD8IL2ofCD8"] = "CD8+ T cell(IL2+)"
feature_names_dict["CD8IL6ofCD8"] = "CD8+ T cell(IL6+)"
feature_names_dict["IL4IL6DPofCD4"] = "CD4+ T cell(IL4+IL6+)"
feature_names_dict["CD86HLADRBofB"] = "B cell(activated)"
feature_names_dict["ASCofB"] = "B cell(antibody-secreting/plasma)"
feature_names_dict["USMemofB"] = "B cell(memory)"
feature_names_dict["CSMemofB"] = "B cell(memory)"
feature_names_dict["NaiveBofB"] = "B cell(naive)"
feature_names_dict["DNofB"] = "B cell(double negative)"
feature_names_dict["IL4IL6DPofCD8"] = "CD8+ T cell(IL4+IL6+)"
feature_names_dict["IL4ofCD8"] = "CD8+ T cell (IL4+)"
feature_names_dict["DC2ofLive"] = "cDC2"
feature_names_dict["CD4IL6ofCD4"] = "CD4+ T cell(IL6+)"
feature_names_dict["TotalMonoofLive"] = "Total monocytes"
feature_names_dict["intMonoofLive"] = "Intermediate monocytes"

################################################
# Load the computed interval information
################################################

Ftest_df = pd.read_csv("./results/MY-LC-Ftest-pvals.csv", index_col = 0)
Scale_df = pd.read_csv("./results/MY-LC-ScaleRobustPALMRT-pvals.csv", index_col = 0)
OLS_df = pd.read_csv("./results/MY-LC-OLSL2PALMRT-intervals.csv", index_col = 0)
Huber_df = pd.read_csv("./results/MY-LC-HuberHuberRobustPALMRT-intervals.csv", index_col = 0)
intervals_info_df = pd.concat([Ftest_df, Scale_df, OLS_df, Huber_df], axis = 1)

In [3]:
print(f"Number of Long-COVID patients: {(df['LC'] == 1).sum()}")
print(f"Number of Control patients: {(df['LC'] == 0).sum()}")

Number of Long-COVID patients: 99
Number of Control patients: 77


In [4]:
Ftest_rejections = set(Ftest_df.index[Ftest_df["Ftest-pval"] <= 0.05])
Scale_rejections = set(Scale_df.index[Scale_df["Scale-pval"] <= 0.05])

In [5]:
Scale_rejections.difference(Ftest_rejections)

{'CD4IFNypofCD4', 'CD4TfhofCD4', 'CD8TNFapofCD8', 'IL4ofCD8'}

In [6]:
Scale_rejections

{'CD4IFNypofCD4',
 'CD4IL2ofCD4',
 'CD4IL4pofCD4',
 'CD4IL6ofCD4',
 'CD4TexofCD4',
 'CD4TfhofCD4',
 'CD86HLADRBofB',
 'CD8IL2ofCD8',
 'CD8IL6ofCD8',
 'CD8TNFapofCD8',
 'DC1ofLive',
 'DNofB',
 'IL4IL6DPofCD4',
 'IL4IL6DPofCD8',
 'IL4ofCD8'}

In [19]:
print(f"Number of rejections by dispersion PALMRT: {len(Scale_rejections)}")

Number of rejections by dispersion PALMRT: 15


In [8]:
Scale_df.loc[list(Scale_rejections.difference(Ftest_rejections))]

,Scale-pval
CD8TNFapofCD8,0.00650
IL4ofCD8,0.00775
CD4TfhofCD4,0.03700
CD4IFNypofCD4,0.00300


In [24]:
print(f"Standard deviation of percent of CD8+ T-cells expressing IL6: {df['CD8IL6ofCD8'].std():.1f}%")
print(f"10th percentile of percent of CD8+ T-cells expressing IL6: {df['CD8IL6ofCD8'].quantile(0.10):.1f}%")
print(f"90th percentile of percent of CD8+ T-cells expressing IL6: {df['CD8IL6ofCD8'].quantile(0.90):.1f}%")

Standard deviation of percent of CD8+ T-cells expressing IL6: 7.8%
10th percentile of percent of CD8+ T-cells expressing IL6: 0.3%
90th percentile of percent of CD8+ T-cells expressing IL6: 11.4%
